In [1]:
import pandas as pd
import nltk
import spacy
import re
import seaborn as sns
import matplotlib.pyplot as plt
import exit

In [2]:
inpath = "output_0211_name_standardized.csv"
inpath = "output_2022-03-12_name_standardized.csv"
inpath = "output_2022-03-12.csv"

df = pd.read_csv(inpath) #.iloc[0:50]

Calculate number of characters and number of words in the body.
Note that there are spelling errors, and words separated by blank spaces that should not be there.
Threads have been removed, but not attachments, although emails with attachments are flagged. Therefore, we could also 
compute statistics just for the emails without attachments, to get more realistic stats. 

In [3]:
bodies = df.Body
bodies[0]

"['Response.pdf', 'Brooks, attached is our response to RFIs 008, 042, 043, and 044.', 'Shane A. Moniz, AIA', 'Architect', 'Conn & Associates, Inc., Architects', '1960-C Buford Boulevard', 'Tallahassee, Florida 32308', 'p 850-878-8784', 'm 850-445-4056', 'smoniz@connandassociates.com', 'NUMBER:', 'REQUEST FOR INFORMATION 003', 'Date: March 2, 2015 PROJECT The Edison Restaurant Cascades Park', 'REFERENCE Project 0500', 'To: Shane Moniz']"

In [4]:
df.shape[0]

71143

In [5]:
exit()

TypeError: 'module' object is not callable

In [ ]:
list(bodies)[0]

In [ ]:
eval(bodies[0])

In [ ]:
nb_words = []
nb_chars = []
body_len = []
body_list = []

# print("bodies columns: ", bodies.columns)
for i, row in enumerate(bodies):
    body = eval(row)
    body = " ".join(body)
    words = body.split(" ")
    text = "".join(words)
    if i < 3: 
        #print("-> row: ", row)
        # print("-> body: ", body)
        print("-> row.Body: ", df.iloc[i].Body)
        print("-> type(body): ", type(body))
    body_list.append(body)
    # print("len(body_list): ", len(body), len(body_list))
    body_len.append(len(body))
    nb_words.append(len(words))
    nb_chars.append(len(text))
len(body_list), len(body_len), len(nb_words), len(nb_chars)

In [ ]:
body[0]

In [ ]:
body_list[0]

In [ ]:
df1 = df.copy()
df1['nb_words'] = nb_words
df1['nb_chars'] = nb_chars
df1['body_len'] = body_len
df1['body'] = body_list  # leaving original Body column

In [ ]:
df1[["body", "Body", "body_len"]]

In [ ]:
df1.to_csv("output_with_attributes.csv", index=0)

In [ ]:
df2 = pd.read_csv("output_with_attributes.csv")

In [ ]:
df2 = df2.fillna('')
df2.isnull()
df2.Importance.iloc[0]

# Remove email duplicates
* Duplications have same From:, Send:, number of chars, number of words

In [ ]:
# All mails with from with a large length have isDisplacement = True
long_count = 0
error_list = []

for item, row in df2.iterrows():
    error_list.append("None")
    # r = eval(row.From)  # Eval no longer required
    r = row.From   #  MUST FIX THIS. NOT WORKING
    disp = row.isDisplacement
    lg = len(r[0]) + len(r[1]) + len(r[2])
    if lg > 200:
        print(row.filenm, lg, disp)
        error_list[-1] = "From"
        long_count += 1
        
print("long_count: ", long_count)
df2['Error_from'] = error_list

In [ ]:
error_sent = [] 
long_count = 0
for item, row in df2.iterrows():
    error_sent.append("None")
    r = row.Sent
    if len(r) > 200:
        error_sent[-1] = True
        long_count += 1
        
df2["Error_sent"] = error_sent

In [ ]:
dfg = df2.groupby(["From", "To", "Sent", "body", "body_len", "nb_words"]).size().to_frame()
#dfg = df2.groupby(["From"]).size()
print(type(dfg), dfg.shape)

In [ ]:
df3 = df2.drop_duplicates(["From", "To", "Sent", "body"], keep='first')
df3.shape

In [ ]:
df3.to_csv("output_with_attributes_no_duplicates.csv", index=0)

In [ ]:
df3 = pd.read_csv("output_with_attributes_no_duplicates.csv")
df3 = df3.fillna('')  # replace NaN by ''

In [ ]:
df3.shape, df1.shape

In [ ]:
df3[df3.Sent.map(len) > 100].Sent.map(len)

In [ ]:
df3.columns

In [ ]:
df3.Importance.head(50)

In [ ]:
df3[df3.Sent.map(len) > 100].isDisplacement

In [ ]:
# Now group emails by sender and calculate statistics
df3g = df3.groupby('From')

In [ ]:
df3.nb_chars.dtype

In [ ]:
dd = df3g['nb_words'].size()
dd

In [ ]:
df3['mn_nb_words'] = df3g['nb_words'].transform('mean')
df3['std_nb_words'] = df3g['nb_words'].transform('std')
df3['mn_nb_chars'] = df3g['nb_chars'].transform('mean')
df3['std_nb_chars'] = df3g['nb_chars'].transform('std')
df3['email_count'] = df3g['nb_words'].transform('size')
df3.columns

In [ ]:
# Confirm that the 'count' columns is the number of emails per sender
df3.groupby('From')['mn_nb_words'].size().values.sum(), df3.shape

In [ ]:
df3[['body_len','nb_chars','nb_words','mn_nb_words','std_nb_words','mn_nb_chars','std_nb_chars','email_count']]

In [ ]:
df3.to_csv("output_with_stats_columns.csv", index=0)

In [ ]:
df3.describe().T

In [ ]:
df3.describe().T

---
# Some plots with Seaborn

In [ ]:

sns.histplot(x="std_nb_words",  data=df3)

In [ ]:
dfgrps = df2.groupby(["From", "body_len"]).groups
keys = list(dfgrps.keys())
k = keys[0]
k[1]
keys_1 = [k for k in keys if k[1] == 1]

In [ ]:
keys_1

In [ ]:
dfg = df2.groupby(["From", "body_len"])

In [ ]:
for k in keys_1:
   print("==> ", dfg.get_group(k).body)

In [ ]:
df.From.to_csv("from.csv", index=0)

In [ ]:
body_list = eval(bodies[0])

In [ ]:
type(bodies[0])

---

In [ ]:
# Number of words. All words that don't relate to the email itself should be removed.
# We'll do that at a later stsage. 


In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
words = [len([i for i in nlp(re.sub('</b>|<b>', '', i2)) if not (i.is_space or i.is_punct)]) for i2 in body]

In [ ]:
words = nlp(body)
body

In [ ]:
nb_words = len(words)
nb_chars = len(body)
nb_words, nb_chars

We must remove punctuation, and special characters. Lookint at the list below, perhaps remove phone numbers, any numbers, hyphens? 
I can also create my own parser? 
Read about [Spacy](https://spacy.io/usage/spacy-101#features).

In [ ]:
words

In [ ]:
for w in words:
    print(w)

In [ ]:
for ent in words.ents:
    print(ent, ",,", ent.text, ent.start_char, ent.end_char, ent.label_)